In [2]:
screen_names = [
    "Recap",
    "Hyperparameter optimization",
    "Expanding grid search",
    "Visualizing hyperparameter values",
    "Varying Hyperparameters",
    "Practice the workflow",
    "Next steps",
    "Takeaways"
]


print('<center><h1>Title</h1></center>')
for i, name in enumerate(screen_names, 1):
    print(f'## {i}. {name}.\n### Exercise')

<center><h1>Title</h1></center>
## 1. Recap.
### Exercise
## 2. Hyperparameter optimization.
### Exercise
## 3. Expanding grid search.
### Exercise
## 4. Visualizing hyperparameter values.
### Exercise
## 5. Varying Hyperparameters.
### Exercise
## 6. Practice the workflow.
### Exercise
## 7. Next steps.
### Exercise
## 8. Takeaways.
### Exercise


<center><h1>Hyperparameter Optimization</h1></center>

## 1. Recap.
### Exercise
## 2. Hyperparameter optimization.
### Exercise
## 3. Expanding grid search.
### Exercise
## 4. Visualizing hyperparameter values.
### Exercise
## 5. Varying Hyperparameters.
### Exercise
## 6. Practice the workflow.
### Exercise
## 7. Next steps.
### Exercise
## 8. Takeaways.
### Exercise